# Quantum registers
NOTE: this notebook follows the contents of chapter 03 of **Mastering Quantum Computing with IBM QX**. The original code is available at https://github.com/PacktPublishing/Mastering-Quantum-Computing-with-IBM-QX

A quantum register is a superposition of n qubits. While a n-bits classical register can store only one value at a time among the $2^n$ possible states, a **quantum register can store any linear combination of the those states**.
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$

In [10]:
from math import log

import numpy as np
from functools import reduce
from itertools import product

from core.qc import *

A single qubit lies in a Hilbert space $\mathcal{H}_{1}$ whose orthonormal basis is {$\ket{{0}}$, $\ket{1}$}.

A quantum register made up of 2 qubits lies in the Hilbert $\mathcal{H}$ given by the tensor product of the spaces of the composing qubits: $\mathcal{H} = \mathcal{H}_{1} \otimes \mathcal{H}_{2}$. The basis of the new space is {$\ket{00}$, $\ket{01}$, $\ket{10}$, $\ket{11}$}.

In [11]:
def create_quantum_state(qubits):
    return reduce(lambda x,y: np.kron(x, y), qubits)

In [12]:
# create a quantum register from |0> and |1>
reg_01 = create_quantum_state([zero_qubit, one_qubit])
print(reg_01)

[0 1 0 0]


In [13]:
reg_four_qubits = create_quantum_state([one_qubit, zero_qubit, one_qubit, zero_qubit])
print(reg_four_qubits)

[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]


In [14]:
reg_plus_minus = create_quantum_state([plus_qubit, minus_qubit])
print(reg_plus_minus)

reg_plus_plus = create_quantum_state([plus_qubit, plus_qubit])
print(reg_plus_plus)

[ 0.5 -0.5  0.5 -0.5]
[0.5 0.5 0.5 0.5]


## Separable states
The following function tries to guess which qubits generated a certain quantum state.

In [15]:
def guess_qubits(quantum_state):
    v = [zero_qubit, one_qubit, plus_qubit, minus_qubit, cw_qubit, ccw_qubit]
    n_qubits = int(log(quantum_state.size, 2))
    
    # product computes the cartesian product of the input iterables
    # product(v, repeat=3) is the same as product(v, v, v)
    for qubits in product(v, repeat=n_qubits):
        guess = create_quantum_state(qubits)
        
        # check if the guessed state and the input are element-wise equal
        if np.allclose(guess, quantum_state):
            return qubits


print(guess_qubits(reg_01))
print(guess_qubits(reg_plus_plus))
print(guess_qubits(reg_plus_minus))

(array([1, 0]), array([0, 1]))
(array([0.70710678, 0.70710678]), array([0.70710678, 0.70710678]))
(array([0.70710678, 0.70710678]), array([ 0.70710678, -0.70710678]))


## Entangled state

Some quantum states can not be separated following this procedure without any modifications. An improvement could be obtained by increasing the elements in the list $v$. This is not, however, a feasible path due to the computational costs.
A non-separable state is known as **entangled state**.

One example of an entangled state is $\ket{\psi} = \frac{\ket{00} + \ket{11}}{\sqrt{2}}$.

In [7]:
psi = 1/np.sqrt(2) * (create_quantum_state([zero_qubit, zero_qubit]) + create_quantum_state([one_qubit, one_qubit]))

print(psi)
print(guess_qubits(psi))

[0.70710678 0.         0.         0.70710678]
None


## Quantum measurements

In [8]:
from random import random

def measure(state):
    n = int(log(state.size, 2))
    
    # element-wise product
    probabilities = state.conj() * state
    
    rand = random()
    for idx, realization in enumerate(product([0, 1], repeat=n)):
        if rand < sum(probabilities[0:(idx+1)]):
            return "|" + "".join(map(str, realization)) + ">"


print(measure(reg_four_qubits))

|1010>


The _rand_ variable in the measure function introduces a source of randomness in the result of the measure. This is not evident in the _reg_four_qubits_ since only one state is possible with probability exactly 1.0.

Insted if we measure the $\ket{\psi}$ state defined before the function will output $\ket{00}$ and $\ket{11}$ with the same probability. The reason is that rand is the realization of a uniform distrubtion over $[0, 1)$: half of the time _rand_ is below $0.5$, the other half is above.

The following cell demonstrantes this behaviour:

In [9]:
def build_psi():
    return 1 / np.sqrt(2) * (create_quantum_state([zero_qubit, zero_qubit]) + 
                             create_quantum_state([one_qubit, one_qubit]))

N = 1000
occurrences = [0, 0]
for _ in range(N):
    if measure(build_psi()) == "|00>":
        occurrences[0] += 1
    else:
        occurrences[1] += 1

print("|00> with probability ", occurrences[0] * 100.0 / N)
print("|11> with probability ", occurrences[1] * 100.0 / N)

|00> with probability  51.0
|11> with probability  49.0


## Decoherence
Two waves are **coherent** if they have the same frequency and a constant phase difference, and the same waveform. The **decoherence** phenomena may alter such properties of a qubit leading to incorrect or unexpected measurements.

- $T_{1}$ is the **relaxation time** and it quantifies how quickly the qubit loses its energy. Given a qubit with initial state $\ket{1}$ the probability of actually measuring 1 is NOT constant in time: P($\ket{1}$) = $e^{\frac{-t}{T_{1}}}$.
- $T_{2}$ is the **dephasing time**. Similarly to $T_{1}$, $T_{2}$ describes the decay from the state $\ket{+}$ to $\ket{-}$. **It is not the time between the two states, but a measure of how quickly the initial and final states become uncorrelated.**